In [6]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [7]:
X, y = mnist["data"], mnist["target"]
X.shape

(70000, 784)

In [101]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
#X_train, X_test, y_train, y_test = X[:1000], X[60000:], y[:1000], y[60000:]

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_neighbors': [3, 5],
    'weights': ['uniform', 'distance']},
  ]

# train across 2 folds, that's a total of 3*2=6 rounds of training 
classifier=KNeighborsClassifier(n_jobs=-1)
grid_search = GridSearchCV(classifier, param_grid, cv=2,
                           scoring='accuracy', return_train_score=True, n_jobs=-1)

#grid_search.fit(X_train, y_train) #expensive operation. I did it on a smaller data scale

GridSearchCV(cv=2, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'n_neighbors': [3, 5],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='accuracy', verbose=0)

In [16]:
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [19]:
classifier=KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=-1)
classifier.fit(X_train, y_train)
classifier.score(X_test, y_test)

0.9717

In [32]:
X_train[0].shape

(784,)

In [28]:
a=np.array([[0,0],[0, 1]])

In [102]:
import numpy as np

def shift(a):
    a=a.reshape(28,28)
    return np.array([
        a.reshape((1, 784)), # original
        np.array(np.roll(a, shift=1, axis=0)).reshape((1, 784)),
        np.array(np.roll(a, shift=-1, axis=0)).reshape((1, 784)),
        np.array(np.roll(a, shift=1, axis=1)).reshape((1, 784)),
        np.array(np.roll(a, shift=-1, axis=1)).reshape((1, 784))
    ])

def shift_y(a):
    return np.array([[a], [a], [a], [a], [a]])
        
# Usually the data is shuffled. In KNeighborsClassifier I don't think it helps.
X_train_augmented=np.apply_along_axis(shift, 1, X_train).reshape(-1, 784)
y_train_augmented=np.apply_along_axis(shift_y, 1, y_train.reshape((-1,1))).reshape(-1)

In [103]:
classifier=KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=-1)
classifier.fit(X_train_augmented, y_train_augmented)
classifier.score(X_test, y_test)

0.9763